# DMPNN

## Featurizing molecules

In [1]:
!git clone https://github.com/deepchem/deepchem.git
!pip install wandb --upgrade

Cloning into 'deepchem'...
remote: Enumerating objects: 54585, done.
remote: Counting objects: 100% (177/177), done.
remote: Compressing objects: 100% (91/91), done.
remote: Total 54585 (delta 103), reused 141 (delta 86), pack-reused 54408
Receiving objects: 100% (54585/54585), 502.64 MiB | 13.50 MiB/s, done.
Resolving deltas: 100% (41375/41375), done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.8 MB 14.7 MB/s 
     |████████████████████████████████| 181 kB 73.7 MB/s 
     |████████████████████████████████| 158 kB 69.1 MB/s 
     |████████████████████████████████| 63 kB 2.0 MB/s 
     |████████████████████████████████| 157 kB 97.7 MB/s 
     |████████████████████████████████| 157 kB 80.7 MB/s 
     |████████████████████████████████| 157 kB 74.8 MB/s 
     |████████████████████████████████| 157 kB 68.7 MB/s 
     |████████████████████████████████| 157 kB 67.4 MB/s 
     |████████████████████

In [2]:
import os
os.chdir(r"deepchem")

In [3]:
!pwd

/content/deepchem


In [4]:
!python setup.py install
!pip install rdkit
!pip install pytorch-lightning

running install
running bdist_egg
running egg_info
creating deepchem.egg-info
writing deepchem.egg-info/PKG-INFO
writing dependency_links to deepchem.egg-info/dependency_links.txt
writing requirements to deepchem.egg-info/requires.txt
writing top-level names to deepchem.egg-info/top_level.txt
writing manifest file 'deepchem.egg-info/SOURCES.txt'
adding license file 'LICENSE'
writing manifest file 'deepchem.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_py
creating build
creating build/lib
creating build/lib/deepchem
copying deepchem/__init__.py -> build/lib/deepchem
creating build/lib/deepchem/rl
copying deepchem/rl/ppo.py -> build/lib/deepchem/rl
copying deepchem/rl/a2c.py -> build/lib/deepchem/rl
copying deepchem/rl/__init__.py -> build/lib/deepchem/rl
creating build/lib/deepchem/models
copying deepchem/models/chemnet_models.py -> build/lib/deepchem/models
copying deepchem/models/text_cnn.py -> build/lib/deepchem/models

In [5]:
from deepchem import deepchem as dc

wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.
wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.


In [6]:
from deepchem.feat import DMPNNFeaturizer

In [7]:
# import deepchem as dc
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# RDKit
import rdkit
from rdkit import Chem
from rdkit.Chem import Draw
from rdkit.Chem import PandasTools
from rdkit.Chem import rdMolDescriptors
from rdkit.Chem.Draw import SimilarityMaps

import os
from pathlib import Path
import csv
import math

In [8]:
content_path = Path('/content/drive/MyDrive/chem_data')
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# !pip install -qU deepchem wandb

In [9]:
import wandb
wandb.login()

import warnings
warnings.filterwarnings('ignore')

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [10]:
from deepchem.models.wandblogger import WandbLogger

In [37]:
# Create directories
# def create_dir():
""" Create directories featurized, valid, train, test.."""


# create_dir()
dmpnn_dir = content_path/'dmpnn_new_dir'
if not dmpnn_dir.is_dir():
  dmpnn_dir.mkdir()

In [12]:
# Featurize data into dc dataset

from deepchem.data import *
def csv_to_feat(filename,):

  data = pd.read_csv(f"{content_path}/{filename}.csv")
  smiles = data['smiles'].tolist()
  pki = data['pki'].tolist()
  featurizer = DMPNNFeaturizer()
  X = featurizer.featurize(smiles)
  dataset = NumpyDataset(X=X, y=pki)
  # print(dataset)
  # print(pki)
  return dataset

In [23]:
# Metrics
from deepchem.metrics import *

metric = Metric(
    mean_absolute_error, task_averager=np.mean, mode="regression")

pearson_r2_score = Metric(
    pearson_r2_score, task_averager=np.mean, mode="regression")

mean_absolute_error = Metric(
    mean_absolute_error, task_averager=np.mean, mode="regression")

rms_score = Metric(
    rms_score, task_averager=np.mean, mode="regression")

### Trial GCNModel

In [14]:
!pip install dgllife
# !pip install dgl
!pip install dgl-cu101 dglgo -f https://data.dgl.ai/wheels/repo.html

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 220 kB 14.8 MB/s 
     |████████████████████████████████| 22.3 MB 270 kB/s 
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.0.2
    Uninstalling scikit-learn-1.0.2:
      Successfully uninstalled scikit-learn-1.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
yellowbrick 1.5 requires scikit-learn>=1.0.0, but you have scikit-learn 0.24.2 which is incompatible.


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://data.dgl.ai/wheels/repo.html
     |████████████████████████████████| 155.6 MB 120.3 MB/s 
     |████████████████████████████████| 63 kB 2.0 MB/s 
     |████████████████████████████████| 281 kB 36.2 MB/s 
     |████████████████████████████████| 45 kB 3.9 MB/s 
     |████████████████████████████████| 78 kB 7.1 MB/s 
     |████████████████████████████████| 103 kB 91.1 MB/s 
     |████████████████████████████████| 51 kB 625 kB/s 
     |████████████████████████████████| 109 kB 88.4 MB/s 
     |████████████████████████████████| 41 kB 452 kB/s 
     |████████████████████████████████| 3.2 MB 78.2 MB/s 
     |████████████████████████████████| 546 kB 78.2 MB/s 
     |████████████████████████████████| 100 kB 10.5 MB/s 
     |████████████████████████████████| 121 kB 89.6 MB/s 
     |████████████████████████████████| 90 kB 11.2 MB/s 
     |████████████████████████████████| 84

In [15]:
from deepchem.models.torch_models import DMPNNModel
from deepchem.models.wandblogger import WandbLogger
from deepchem.models import *
from deepchem.metrics import *

In [35]:
# trial model 



def train_DMPNN(train, val, tid, num_message=3, num_epoch=5):

  # metric = Metric(mean_absolute_error)
  # metric = Metric(
    # mean_absolute_error, task_averager=np.mean, mode="regression")
  # callback = ValidationCallback(val, 500, metric)
  logger = WandbLogger(project='dmpnn_rerun', name=tid)
  model = DMPNNModel(mode='regression', n_tasks=1, 
                    learning_rate=0.001, wandb_logger=logger,
                     enc_hidden=400, depth=4, ffn_dropout_p=0.05)
  loss = model.fit(train, nb_epoch=num_epoch)
  # loss = model.fit(train, nb_epoch=num_epoch, callbacks=callback)
  logger.finish()


  model_mae = model.evaluate(val, [mean_absolute_error], per_task_metrics=False)
  model_r2 = model.evaluate(val, [pearson_r2_score], per_task_metrics=False)
  model_rmse = model.evaluate(val, [rms_score], per_task_metrics=False)

  print(f"mae: {model_mae}")
  print(f"r2: {model_r2}")
  print(f"rmse: {model_rmse}")

  return model
  # return model_mae['mean-mean_absolute_error'], model_r2['mean-pearson_r2_score'], model_rmse['mean-rms_score']

In [19]:
# split dataset
def split_data(dataset):
  from deepchem.splits.splitters import RandomSplitter

  splitter = RandomSplitter()
  train, val, test = splitter.train_valid_test_split(
      dataset=dataset,
      log_every_n=100
  )
  return train, val, test


In [34]:
dataset_ids = ['11', '15', '51', '72', '87', '100', '107', '108', '114', '121', '129', '130', '136', '137', '138', '155', '165', '176', '194', '252', '259', '278', '280', '10142', '10193', '10280', '10627', '11290', '12209', '12952', '19905']
len(dataset_ids)

31

In [18]:
def basic_dmpnn(data):
  tid = 'tid' + data
  dataset = csv_to_feat(tid)
  train, val, test = split_data(dataset)
  model = train_DMPNN(train, val, tid, num_epoch=30)
  model_mae = model.evaluate(test, [mean_absolute_error], per_task_metrics=False)
  model_r2 = model.evaluate(test, [pearson_r2_score], per_task_metrics=False)
  model_rmse = model.evaluate(test, [rms_score], per_task_metrics=False)

  print(f"mae: {model_mae}")
  print(f"r2: {model_r2}")
  print(f"rmse: {model_rmse}")

In [31]:
basic_dmpnn('11')

train/loss,▁▅▄▄▂█▁▂▁▂▁▁▁▁▁▂▁▃▁▁▁▁▁█▂▁▁▃▁▁▃▁▁▁▁▁▁▁▁▂
train/loss,1.54142


mae: {'mean-mean_absolute_error': 0.7044813392100716}
r2: {'mean-pearson_r2_score': 0.5543074457408771}
rmse: {'mean-rms_score': 0.8870461047673646}
mae: {'mean-mean_absolute_error': 0.5181033731707653}
r2: {'mean-pearson_r2_score': 0.6907118998399516}
rmse: {'mean-rms_score': 0.6555866189800114}


In [ ]:
# from deepchem import *
# from deepchem.trans import *

basic_dmpnn('11')

train/loss,█▁▂▁▂▁▁▁▃▁▁▁▂▂▁▁▁▁▁▁▁▁▁▁▁▂▁▂▁▁▁▁▁▁▁▁▁▁▁▁
train/loss,0.0104


mae: {'mean-mean_absolute_error': 0.63167525409677}
r2: {'mean-pearson_r2_score': 0.5588941709224193}
rmse: {'mean-rms_score': 0.8689418924235928}


In [32]:
def run_dmpnn(num_runs, datasets, num_epochs):
  # results = {}
  mean_results = {}

  for id in datasets:
    results = {}

    mae_list = []
    r2_list = []
    rmse_list = []

    tid = 'tid'+id
    dataset = csv_to_feat(tid)
    # train, val, test = split_data(dataset)
    
    for run in range(num_runs):
      train, val, test = split_data(dataset)
      model = train_DMPNN(train, val, tid, num_epochs)
      mae = model.evaluate(test, [mean_absolute_error], per_task_metrics=False)
      r2 = model.evaluate(test, [pearson_r2_score], per_task_metrics=False)
      rmse = model.evaluate(test, [rms_score], per_task_metrics=False)


      print(f"mae: {mae}")
      print(f"r2: {r2}")
      print(f"rmse: {rmse}")
      mae_list.append(mae['mean-mean_absolute_error'])
      r2_list.append(r2['mean-pearson_r2_score'])
      rmse_list.append(rmse['mean-rms_score'])

    
    results[f'{tid}_mae'] = mae_list
    results[f'{tid}_r2'] = r2_list
    results[f'{tid}_rmse'] = rmse_list

    mean_mae = np.mean(mae_list)
    mean_r2 = np.mean(r2_list)
    mean_rmse = np.mean(rmse_list)

    mean_results[f'{tid}_mean_mae'] = mean_mae
    mean_results[f'{tid}_mean_r2'] = mean_r2
    mean_results[f'{tid}_mean_rmse'] = mean_rmse


    print(f'mean mae for {num_runs} for {tid}: {mean_mae}')
    print(f'mean r2 for {num_runs} for {tid}: {mean_r2}')
    print(f'mean rmse for {num_runs} for {tid}: {mean_rmse}')
   
    results_df = pd.DataFrame(results)
    results_df.to_csv(f"{dmpnn_dir}/{tid}_dmpnn.csv")

  return results, mean_results


In [38]:
dmpnn_results, mean_dmpnn_results = run_dmpnn(2, dataset_ids, 35)

train/loss,▃▁▁▂▁▂▁▁▇▁▁▁▁▁▁▂▁▂▁▁▂█▂▁▂▁
train/loss,0.88292


mae: {'mean-mean_absolute_error': 0.9352570749638346}
r2: {'mean-pearson_r2_score': 0.225248419262265}
rmse: {'mean-rms_score': 1.2787866478908716}
mae: {'mean-mean_absolute_error': 0.7764041257209838}
r2: {'mean-pearson_r2_score': 0.48824249466034947}
rmse: {'mean-rms_score': 1.0821742190952666}


train/loss,▁▁▁▁█▁▁▂▁▃▁▂▂▁▂▂▁▁▁▁▂▁▁▁▁▁
train/loss,0.41762


mae: {'mean-mean_absolute_error': 0.7105840976732251}
r2: {'mean-pearson_r2_score': 0.36447896465790014}
rmse: {'mean-rms_score': 0.9354947129759398}
mae: {'mean-mean_absolute_error': 0.7372900310284896}
r2: {'mean-pearson_r2_score': 0.279458362250695}
rmse: {'mean-rms_score': 0.9648976457550632}
mean mae for 2 for tid11: 0.7568470783747367
mean r2 for 2 for tid11: 0.3838504284555222
mean rmse for 2 for tid11: 1.023535932425165


train/loss,▁▂▁▁▂▄▅▁▁▂▂▃▁▄▂▂▂▃▂▁▁▇▁▂▁▂▁▁▇█▁▂▆▁▃▁▁▁▁▁
train/loss,0.07712


...

mae: {'mean-mean_absolute_error': 0.8501806827469667}
r2: {'mean-pearson_r2_score': 0.3327770601436867}
rmse: {'mean-rms_score': 1.0378836683389898}
mae: {'mean-mean_absolute_error': 0.7610361798585911}
r2: {'mean-pearson_r2_score': 0.4785584781427277}
rmse: {'mean-rms_score': 0.8961106603747514}
mean mae for 2 for tid12952: 0.732833218336185
mean r2 for 2 for tid12952: 0.48894571104105755
mean rmse for 2 for tid12952: 0.8826977347766327


train/loss,▁▃▆▁▂▁▂▂▁█▂▃▂▅▁▂▂▂▃▁▁▁▂▁▅▁▄▁
train/loss,0.00131


mae: {'mean-mean_absolute_error': 0.8359140020508992}
r2: {'mean-pearson_r2_score': 0.12133427692245367}
rmse: {'mean-rms_score': 0.9928589425122336}
mae: {'mean-mean_absolute_error': 0.9497310316106383}
r2: {'mean-pearson_r2_score': 0.12030352283176785}
rmse: {'mean-rms_score': 1.0677185572910557}


train/loss,▁█▄▁▁▃▂▂▁▂▁▁▁▁▁▆▃▂▂▁▁▂▁▂▁▁▂▃
train/loss,2.7157


mae: {'mean-mean_absolute_error': 0.8381748059115702}
r2: {'mean-pearson_r2_score': 0.21550312704756752}
rmse: {'mean-rms_score': 1.0687087722091881}
mae: {'mean-mean_absolute_error': 0.7550663909325408}
r2: {'mean-pearson_r2_score': 0.2185055652527597}
rmse: {'mean-rms_score': 1.0557943735412407}
mean mae for 2 for tid19905: 0.8523987112715896
mean r2 for 2 for tid19905: 0.16940454404226377
mean rmse for 2 for tid19905: 1.0617564654161482


In [ ]:
dmpnn_results, mean_dmpnn_results = run_dmpnn(3, dataset_ids, 50)


train/loss,▂▂▁▄▁▁▂▁▁▂▂▃▁▂▅▁▁▁▃▁▂█▁▁▁▃
train/loss,1.89019


mae: {'mean-mean_absolute_error': 0.8417591413275523}
r2: {'mean-pearson_r2_score': 0.4481636348929503}
rmse: {'mean-rms_score': 1.060414425465403}


train/loss,▁▁▂▃▅▂▁▃▁▂█▁▆▁▁▂▂▁▁▂▁▁▁▃▂▁
train/loss,0.16389


mae: {'mean-mean_absolute_error': 0.7689535810483097}
r2: {'mean-pearson_r2_score': 0.3797268971734641}
rmse: {'mean-rms_score': 0.9491552814493082}


train/loss,▁▁█▅▂▁▁▂▂▁▁▁▁▁▁▁▂▁▁▂▂▁▁▁▁▁
train/loss,0.39512


...

mae: {'mean-mean_absolute_error': 0.7088590244332557}
r2: {'mean-pearson_r2_score': 0.3341847735099305}
rmse: {'mean-rms_score': 0.9129165132625185}


train/loss,▁▂▂▁▂▃▁▁▂▁▂▁▂█▂▂▁▁▂▂▂▁▂▁▁▁▁▁
train/loss,0.16757


mae: {'mean-mean_absolute_error': 0.5801013570534016}
r2: {'mean-pearson_r2_score': 0.273277503662647}
rmse: {'mean-rms_score': 0.7450661237799734}


train/loss,▃▁▃▁▁▃▁▁▁▂▂▆▁▆▁▁▁▁▇█▁▁▂▄▁▂▁▂
train/loss,0.29127


mae: {'mean-mean_absolute_error': 0.7588644506918695}
r2: {'mean-pearson_r2_score': 0.16041097657150252}
rmse: {'mean-rms_score': 0.9336469002665919}
mean mae for 3 for tid19905: 0.6826082773928422
mean r2 for 3 for tid19905: 0.25595775124802667
mean rmse for 3 for tid19905: 0.8638765124363612


In [ ]:
basic_gcn_results, mean_basic_gcn_results = run_mpnn(3, dataset_ids, 50)

In [ ]:
!pip install --pre deepchem

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from deepchem.feat import DMPNNFeaturizer

### Hyperparameter

In [ ]:
# Hyperparameters tuning
!pip install pyGPGO
import pyGPGO
from pyGPGO.covfunc import matern32
from pyGPGO.acquisition import Acquisition
from pyGPGO.surrogates.GaussianProcess import GaussianProcess
from pyGPGO.GPGO import GPGO



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.8 MB 7.8 MB/s 
     |████████████████████████████████| 872 kB 52.5 MB/s 
  Created wheel for pyGPGO: filename=pyGPGO-0.5.1-py3-none-any.whl size=19879 sha256=e1cb3311b29194194b3b68080974dcb3bc7f77e93b3b038e34108e63c843174d
  Stored in directory: /root/.cache/pip/wheels/c8/5d/0b/2160114e2f1b87791c51b66cf07f89831dbb6f49167950316f
  Created wheel for Theano-PyMC: filename=Theano_PyMC-1.1.2-py3-none-any.whl size=1529963 sha256=ac26e9dfc315cdae40b00792a04b899ee65cdef7a2ac13ee6e2a695e1bc7f93c
  Stored in directory: /root/.cache/pip/wheels/f3/af/8c/5dd7553522d74c52a7813806fc7ee1a9caa20a3f7c8fd850d5
Successfully built pyGPGO Theano-PyMC


In [ ]:
from deepchem.hyper import *

In [ ]:
params_dict = {
'dense_layer_size': ('int', [16, 2048]),
'graph_conv_layer_size': ('int', [32, 2048]),
'dropout': ('cont', [.0 ,0.5]),
'batch_size': ('int', [10,100]),
'learning_rate': ('cont', [0.0001, 0.0020]),
'nb_epoch': ('int', [20, 200])
}

In [ ]:
params = {
    'enc_hidden': [300, 400, 500],
    'depth': [3, 4, 5],
    'enc_dropout_p': [0.0, 0.01, 0.05],
    'ffn_hidden': [300, 400, 500],
    'ffn_dropout_p': [0.01, 0.0, 0.05]
}

In [ ]:
dataset = csv_to_feat('tid11')
train, val, test = split_data(dataset)

In [ ]:
optimizer = GridHyperparamOpt(DMPNNModel)
best_model, best_hyperparams, all_results = optimizer.hyperparam_search(params_dict=params, 
                                                                        train_dataset=train, valid_dataset=val, metric=metric, nb_epoch=30)

In [ ]:
print(best_hyperparams)

{'enc_hidden': 400, 'depth': 4, 'enc_dropout_p': 0.0, 'ffn_hidden': 300, 'ffn_dropout_p': 0.05}


In [ ]:
import pprint as pp
pp.pprint(all_results)

{'_depth_3_enc_dropout_p_0.000000_enc_hidden_300_ffn_dropout_p_0.000000_ffn_hidden_300': 0.7870639838002165,
 '_depth_3_enc_dropout_p_0.000000_enc_hidden_300_ffn_dropout_p_0.000000_ffn_hidden_400': 0.8082433853947897,
 '_depth_3_enc_dropout_p_0.000000_enc_hidden_300_ffn_dropout_p_0.000000_ffn_hidden_500': 0.7026819684291538,
 '_depth_3_enc_dropout_p_0.000000_enc_hidden_300_ffn_dropout_p_0.010000_ffn_hidden_300': 0.7255186337962786,
 '_depth_3_enc_dropout_p_0.000000_enc_hidden_300_ffn_dropout_p_0.010000_ffn_hidden_400': 0.7381334835497928,
 '_depth_3_enc_dropout_p_0.000000_enc_hidden_300_ffn_dropout_p_0.010000_ffn_hidden_500': 0.8045797229752988,
 '_depth_3_enc_dropout_p_0.000000_enc_hidden_300_ffn_dropout_p_0.050000_ffn_hidden_300': 0.7288426715228012,
 '_depth_3_enc_dropout_p_0.000000_enc_hidden_300_ffn_dropout_p_0.050000_ffn_hidden_400': 0.7225635306806495,
 '_depth_3_enc_dropout_p_0.000000_enc_hidden_300_ffn_dropout_p_0.050000_ffn_hidden_500': 0.7076258730201591,
 '_depth_3_enc_drop

In [ ]:
df_param = pd.DataFrame.from_dict(params_dict,orient='index', )
display(df_param)

In [ ]:
def hyper_model(dense_layer_size, graph_conv_layer_size, batch_size, dropout, nb_epoch, **params):
     
    global neg_mae
    global mae
    global r2
    global rmse
    global config
    
    dense_layer_size = int(round(dense_layer_size))
    graph_conv_depth = 2 # two graph convolutional layers
    graph_conv_layer_size = int(round(graph_conv_layer_size))
    batch_size = int(round(batch_size))
    dropout = dropout
    model = dc.models.GraphConvModel(
            len(tasks),
            batch_size=batch_size,
            dropout=dropout,
            mode='regression',
            graph_conv_layers=[graph_conv_layer_size]*graph_conv_depth,
            dense_layer_size=dense_layer_size,
            # configproto=config,
            **params
            )
    model.fit(train_gcn)

    # MAE
    valid_scores_mae = model.evaluate(val_gcn, [mean_absolute_error], per_task_metrics=False)
    # R2
    valid_scores_r2 = model.evaluate(val_gcn, [pearson_r2_score], per_task_metrics=False)
    # RMSE
    valid_scores_rmse = model.evaluate(val_gcn, [rms_score], per_task_metrics=False)
    # -MAE
    neg_valid_scores_mae = -valid_scores_mae['mean-mean_absolute_error']

    neg_mae.append(neg_valid_scores_mae)
    mae.append(valid_scores_mae['mean-mean_absolute_error'])
    r2.append(valid_scores_r2['mean-pearson_r2_score'])
    rmse.append(valid_scores_rmse['mean-rms_score'])
    
    return neg_valid_scores_mae

In [ ]:
# config = tf.compat.v1.ConfigProto()
# config = tf.ConfigProto()
# config.gpu_options.allow_growth = True

# Run the model 

neg_mae = []
mae = []
r2  = []
rmse = []
cov = matern32()
gp  = GaussianProcess(cov)
acq = Acquisition(mode='ExpectedImprovement')
tasks = ['pki']

gpgo = GPGO(gp, acq, hyper_model, params_dict)

# Adjust the number of iterations, e.g., max_iter=100 and init_eval=10.
max_iter = 10
init_evals = 1
gpgo.run(max_iter=max_iter, init_evals=init_evals)

In [ ]:
# hyperparameter tuning
"""
from deepchem.models import GCNModel

def hyper_model(dense_layer_size, graph_conv_layer_size, batch_size, dropout, nb_epoch, **params):
     
    global neg_mae
    global mae
    global r2
    global rmse
    global config
    
    dense_layer_size = int(round(dense_layer_size))
    graph_conv_depth = 2 # two graph convolutional layers
    graph_conv_layer_size = int(round(graph_conv_layer_size))
    batch_size = int(round(batch_size))
    dropout = dropout
    model = GCNModel(
            n_tasks=1,
            graph_conv_layers=[graph_conv_layer_size]*graph_conv_depth,
            batch_size=batch_size,
            dropout=dropout,
            mode='regression',
            dense_layer_size=dense_layer_size,
            configproto=config,
            **params
            )
    model.fit(train)

    # MAE
    valid_scores_mae = model.evaluate(val, [mean_absolute_error], per_task_metrics=False)
    # R2
    valid_scores_r2 = model.evaluate(val, [pearson_r2_score], per_task_metrics=False)
    # RMSE
    valid_scores_rmse = model.evaluate(val, [rms_score], per_task_metrics=False)
    # -MAE
    neg_valid_scores_mae = -valid_scores_mae['mean-mean_absolute_error']

    neg_mae.append(neg_valid_scores_mae)
    mae.append(valid_scores_mae['mean-mean_absolute_error'])
    r2.append(valid_scores_r2['mean-pearson_r2_score'])
    rmse.append(valid_scores_rmse['mean-rms_score'])
    
    return neg_valid_scores_mae
"""